In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import keras

Using TensorFlow backend.


In [2]:
data=pd.read_csv(r'C:\Users\vikil\Desktop\ML\data_Sets\Churn_Modelling.csv')
data['Exited']=data.Exited.astype(float)
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1.0
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0.0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1.0
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0.0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0.0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0.0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1.0
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1.0


In [15]:
data_in=data.iloc[:,3:13]
data_out=data.iloc[:,13]

In [16]:
#from sklearn.preprocessing import LabelEncoder
#con= LabelEncoder()
#data_geo=con.fit_transform(data_in["Geography"])


In [17]:
geo=pd.get_dummies(data_in['Geography'],drop_first=True)
gender=pd.get_dummies(data_in['Gender'],drop_first=True)

In [18]:
data_in=data_in.drop(['Geography'],axis=1)
data_in=data_in.drop(['Gender'],axis=1)

In [19]:
data_in=pd.concat([data_in,geo,gender],axis=1)
data_in

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,0,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1


In [9]:
from sklearn.model_selection import train_test_split
training,testing= train_test_split(data_in,test_size=0.2,random_state=0)
training_op,testing_op= train_test_split(data_out,test_size=0.2,random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
training=sc.fit_transform(training)
testing=sc.transform(testing)

In [11]:
from sklearn.linear_model import LogisticRegression
clf_log=LogisticRegression()
clf_log.fit(training,training_op)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
pred=clf_log.predict(testing)
pred

array([0., 0., 0., ..., 0., 0., 0.])

In [17]:
testing_op=np.array(testing_op)
testing_op

array([0., 1., 0., ..., 0., 0., 0.])

In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(pred,testing_op)

0.811

In [63]:
from keras.models import Sequential
from keras.layers import Dense
#from keras.layers import LeakyReLU,PReLU,ELU
#from keras.layers import Dropout

In [64]:
classifier=Sequential()

In [65]:
classifier.add(Dense(units=6, kernel_initializer='he_uniform', activation='relu',input_dim=11))

In [66]:
classifier.add(Dense(units=6, kernel_initializer='he_uniform', activation='relu'))

In [67]:
classifier.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))

In [68]:
classifier.compile(optimizer='Adamax',loss='binary_crossentropy',metrics=['accuracy'])

In [69]:
model=classifier.fit(training,training_op,validation_split=0.33,batch_size=10,nb_epoch=100)

Train on 5359 samples, validate on 2641 samples
Epoch 1/100
5359/5359 [==============================] - 3s 625us/step - loss: 0.5060 - accuracy: 0.7794 - val_loss: 0.4817 - val_accuracy: 0.7929
Epoch 2/100
5359/5359 [==============================] - 3s 469us/step - loss: 0.4632 - accuracy: 0.7983 - val_loss: 0.4606 - val_accuracy: 0.7952
Epoch 3/100
5359/5359 [==============================] - 3s 482us/step - loss: 0.4453 - accuracy: 0.8026 - val_loss: 0.4481 - val_accuracy: 0.8035
Epoch 4/100
5359/5359 [==============================] - 3s 480us/step - loss: 0.4347 - accuracy: 0.8097 - val_loss: 0.4412 - val_accuracy: 0.8039
Epoch 5/100
5359/5359 [==============================] - 3s 480us/step - loss: 0.4280 - accuracy: 0.8141 - val_loss: 0.4369 - val_accuracy: 0.8020
Epoch 6/100
5359/5359 [==============================] - 3s 492us/step - loss: 0.4233 - accuracy: 0.8177 - val_loss: 0.4337 - val_accuracy: 0.8042
Epoch 7/100
5359/5359 [==============================] - 3s 501us/step

In [77]:
pred_ann=classifier.predict(testing)
pred_ann=(pred_ann>0.5)
pred_ann.dtype

dtype('bool')

In [75]:
testing_op=(testing_op>0.5)
print(testing_op)

9394    False
898      True
2398    False
5906    False
2343    False
        ...  
1037    False
2899    False
9549    False
2740    False
6690    False
Name: Exited, Length: 2000, dtype: bool


In [76]:
from sklearn.metrics import accuracy_score
accuracy_score(pred_ann,testing_op)

0.8625